In [7]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.2.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Vine_Review').getOrCreate()

In [9]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Lawn_and_Garden_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32787517| RED72VWWCOS7S|B008HDQYLQ|     348668413|Garden Weasel Gar...| Lawn and Garden|          1|            2|          8|   N|                Y|            One Star|I don't hate the ...| 2015-08-31|
|         US|   16374060| RZHWQ208LTEPV|B005OBZBD6|     264704759|10 Foot Mc4 Solar...| Lawn and Garden|          5|    

In [10]:
# create the vine_df
vine_df = df.filter('total_votes >= 20').select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.orderBy(df['total_votes'].asc()).show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1DP8U8LL9PF2M|          5|           17|         20|   N|                Y|
|R33PRRPXLYS2NU|          5|           18|         20|   N|                Y|
| R2X24G7VHEZMN|          3|           18|         20|   N|                Y|
|R1IIU852I1WEJ7|          2|           19|         20|   N|                Y|
| RKFVA45HVD2AM|          1|           17|         20|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [11]:
helpful_votes_df = vine_df.filter((vine_df['helpful_votes']/vine_df['total_votes']) >= 0.5)
helpful_votes_df.orderBy(vine_df['helpful_votes'].asc()).show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1TWE2G585COIU|          1|           10|         20|   N|                N|
|R13J25SOERDKF8|          3|           10|         20|   N|                N|
|R2F3YFRMO03OO4|          2|           10|         20|   N|                Y|
| RK7TR971BFNF4|          1|           10|         20|   N|                Y|
|R2PVU5EB8539RS|          1|           10|         20|   N|                N|
| RNXLGB559GCX3|          1|           10|         20|   N|                Y|
|R3O3IASO1SDB6X|          1|           10|         20|   N|                Y|
| RCCUE5C2NMFAD|          1|           10|         20|   N|                Y|
|R1NH0RPDS9Y1XJ|          1|           10|         20|   N|                N|
|R2IESP4VWIRA5Q|          1|           10|         20|   N|     

In [12]:
vine_review_df = helpful_votes_df.filter(helpful_votes_df['vine'] == 'Y')
vine_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R28DXTC3JQ9IY1|          4|           24|         26|   Y|                N|
|R3AFZKLQXATHBU|          5|           44|         49|   Y|                N|
|R2RUUF2JPJPC0E|          4|           20|         22|   Y|                N|
| RFZ2WUH4248AB|          2|           26|         27|   Y|                N|
|R1Q4LVHIFOWYFR|          5|           23|         28|   Y|                N|
| R4YEPTQED3X1Q|          5|           19|         20|   Y|                N|
|R2Z7C8YCRSC9DP|          5|           22|         22|   Y|                N|
|R3J8OI5CB74P5K|          1|           22|         25|   Y|                N|
| RH39LMKN6AZDC|          5|           33|         40|   Y|                N|
| R8RD8K0ESJSRD|          5|           21|         21|   Y|     

In [13]:
unpaid_review_df = helpful_votes_df.filter(helpful_votes_df['vine'] == 'N')
unpaid_review_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQQ3KVTU5TJ4I|          5|           24|         29|   N|                Y|
|R3FELXWV9T5CWE|          2|           22|         24|   N|                Y|
| ROBYK6EZYK398|          5|           29|         30|   N|                Y|
|R2YVBBR6NXIA4V|          5|           25|         28|   N|                N|
|R2AVTBDIVG2AW4|          5|           26|         26|   N|                N|
|R1Z2LNN3FANMTO|          1|           20|         24|   N|                N|
|  RLNULBKRWNNR|          5|           42|         43|   N|                Y|
| R9QNQUL94RX1F|          3|           27|         33|   N|                Y|
| RTULFZTUS1VBP|          5|           51|         52|   N|                Y|
|R1BM9RBQWI62O2|          5|           43|         60|   N|     

In [21]:
vine_count = vine_review_df.count()
unpaid_count = unpaid_review_df.count()
print(vine_count)
print(unpaid_count)

386
48717


In [22]:
#vine_5stars = vine_review_df.groupby('star_rating').agg({'star_rating':'count'})
#vine_5stars.show()

vine_5stars = vine_review_df[vine_review_df['star_rating'] == 5].count()
unpaid_5stars = unpaid_review_df[unpaid_review_df['star_rating'] == 5].count()
print(vine_5stars)
print(unpaid_5stars)

176
24026


In [24]:
percentage_5star_vines = vine_5stars / vine_count * 100
percentage_5star_unpaid = unpaid_5stars / unpaid_count * 100
print(percentage_5star_vines)
print(percentage_5star_unpaid)

45.59585492227979
49.317486708951705
